## Import

In [1]:
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from tqdm.auto import tqdm

In [2]:
import sys
sys.path.append('/home/sardarchitect/repos/github.com/ucsd_capstone/')
sys.path.append('/home/sardarchitect/repos/github.com/ucsd_capstone/pedtracker/')

In [3]:
from pedtracker.virat import virat
from pedtracker.model import model

## Hyperparameters

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32

## Data Ingestion

In [5]:
video_df = virat.build_virat()
video_df.head()

,path,num_frames,duration,event_file,object_file,mapping_file
name,,,,,,
VIRAT_S_000001,/mnt/d/data/virat/videos/VIRAT_S_000001.mp4,20655,689,VIRAT_S_000001.viratdata.events.txt,VIRAT_S_000001.viratdata.objects.txt,VIRAT_S_000001.viratdata.mapping.txt
VIRAT_S_000002,/mnt/d/data/virat/videos/VIRAT_S_000002.mp4,9075,302,VIRAT_S_000002.viratdata.events.txt,VIRAT_S_000002.viratdata.objects.txt,VIRAT_S_000002.viratdata.mapping.txt
VIRAT_S_000003,/mnt/d/data/virat/videos/VIRAT_S_000003.mp4,20940,698,VIRAT_S_000003.viratdata.events.txt,VIRAT_S_000003.viratdata.objects.txt,VIRAT_S_000003.viratdata.mapping.txt
VIRAT_S_000004,/mnt/d/data/virat/videos/VIRAT_S_000004.mp4,17640,588,VIRAT_S_000004.viratdata.events.txt,VIRAT_S_000004.viratdata.objects.txt,VIRAT_S_000004.viratdata.mapping.txt
VIRAT_S_000006,/mnt/d/data/virat/videos/VIRAT_S_000006.mp4,17805,594,VIRAT_S_000006.viratdata.events.txt,VIRAT_S_000006.viratdata.objects.txt,VIRAT_S_000006.viratdata.mapping.txt


In [6]:
current_video = video_df.loc["VIRAT_S_010207_08_001308_001332"]
current_video

path            /mnt/d/data/virat/videos/VIRAT_S_010207_08_001...
num_frames                                                    429
duration                                                       17
event_file      VIRAT_S_010207_08_001308_001332.viratdata.even...
object_file     VIRAT_S_010207_08_001308_001332.viratdata.obje...
mapping_file    VIRAT_S_010207_08_001308_001332.viratdata.mapp...
Name: VIRAT_S_010207_08_001308_001332, dtype: object

In [7]:
annotations_df = virat.get_annotations(current_video['path'])
annotations_df = annotations_df[annotations_df['object_type'] == 1]
annotations_df.head()

,object_id,object_duration,current_frame,bbox_lefttop_x,bbox_lefttop_y,bbox_width,bbox_height,object_type
0,0,244,185,0,304,10,37,1
1,0,244,186,0,304,12,37,1
2,0,244,187,0,304,14,37,1
3,0,244,188,0,304,16,37,1
4,0,244,189,1,304,16,37,1


## Model Architecture

In [8]:
clf = model.yolov5()
clf.conf = 0.25
clf.classes = [0]

Using cache found in /home/sardarchitect/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-30 Python-3.11.3 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)



requirements: /home/sardarchitect/.cache/torch/hub/requirements.txt not found, check failed.


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


## Evaluation Criteria

In [33]:
num_frames = int(current_video['num_frames'])
capture = cv.VideoCapture(current_video['path'])

preds_list = []

for current_frame in tqdm(range(num_frames)):
    ret, frame = capture.read()
    preds = clf(frame)
    preds_list.append(preds.xyxy)
    
capture.release()

  0%|          | 0/429 [00:00<?, ?it/s]

In [34]:
current_frame = 380

In [54]:
bbox_at_current_frame = []
for bbox in annotations_df[annotations_df['current_frame'] == current_frame].iterrows():
    bbox_x1 = bbox[1]['bbox_lefttop_x']
    bbox_y1 = bbox[1]['bbox_lefttop_y']
    bbox_x2 = bbox[1]['bbox_lefttop_x'] + bbox[1]['bbox_width']
    bbox_y2 = bbox[1]['bbox_lefttop_y'] + bbox[1]['bbox_height']
    bbox_at_current_frame.append([bbox_x1, bbox_y1, bbox_x2, bbox_y1])
np.array(bbox_at_current_frame)

array([[300, 270, 328, 270],
       [264, 236, 289, 236],
       [190, 232, 211, 232],
       [170, 227, 190, 227],
       [113, 222, 131, 222]])

In [63]:
bbox_1 = bbox_at_current_frame[0]
bbox_2 = preds_list[current_frame][0].cpu().numpy()[0][:4]

In [67]:
def mean_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = (xB - xA) * (yB - yA)

    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [ ]:
for 

In [68]:
mean_intersection_over_union(bbox_1, bbox_2)

-0.05457308237786507

## Optimization

## Training Loop

## Testing Loop

## Model Export